In [108]:
# se importan las librerias a utilizar en el proceso de ETL
import pandas as pd
import numpy as np
import pickle
import joblib

In [5]:
# Se carga en la variable df_movies utilizando pandas, la informacion contenida en el archivo movies_dataset.csv
df_movies = pd.read_csv('../Dataset/movies_dataset.csv',delimiter=',')

# Muestro la primera fila del dataframe df_movies
df_movies.head()

c:\Users\Guille\anaconda3\envs\render\lib\site-packages\IPython\core\interactiveshell.py:3552: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
df_shape = df_movies.shape
print(df_shape[0],' filas ')
print(df_shape[1],' columnas')

45466  filas 
24  columnas


In [7]:
# Utilizo la funcion isnull() para detectar valores vacios y luego la funcion sum() para sumar la cantidad de valores vacios segun la columna.
df_movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [8]:
# Consulto el tipo de dato de cada columna.
df_movies.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

CAMBIOS OBLIGATORIOS

In [9]:
# Se crea una variable del tipo lista llamada 'columns' en la cual se carga los nombres de las columnas a tratar en el siguente codigo.
columns = ['revenue','budget']

# se utiliza la funcion fillna() para reemplazar los valores nulos por el numero entero 0.
df_movies[columns] = df_movies[columns].fillna(0)

In [10]:
df_movies.dropna(subset=['release_date'], inplace=True)

In [11]:
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

ValueError: time data 1 doesn't match format specified

Segun el error lanzado se deduce que la columna 'release_date' posee enteros

In [12]:
def clean(df,column):
    formato_comun = df[column].str.isdigit().mean() > 0.5  # Ejemplo: si más del 50% de los valores son dígitos

    # Filtrar los valores que no tienen el formato común
    valores_atipicos = df_movies.loc[~df[column].str.isdigit() if formato_comun else df[column].str.isdigit()]

    # Imprimir los valores atípicos
    return valores_atipicos[column].tolist()

anomalies = clean(df_movies,'release_date')
anomalies

['1', '12', '22']

In [13]:
# utiliza la lista de anomalias 'anomalies'para eliminar las filas afectadas
for a in anomalies:
    df_movies = df_movies.drop(df_movies[df_movies['release_date'] == a].index)

In [14]:
# Transforma el tipo de dato 'object' a tipo 'datetime' con el formato AAAA/MM/DD
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'], format='%Y-%m-%d')

In [15]:
# crea una nueva columna 'release_year' donde carga solo el año
df_movies['release_year'] = df_movies['release_date'].dt.year

In [16]:
# Cambia el tipo de dato 'object' a 'float' de las columnas 'revenue' y 'budget'
list_convert = ['revenue','budget']
df_movies[list_convert] = df_movies[list_convert].astype(float)

In [17]:
# Crea una nueva columna llamanda 'return' donde carga el resultado de la divicion entre las columnas 'revenue' y 'budget', 
# utilizando la funcion where de numpy para condicionar el resultado
df_movies['return'] = np.where((df_movies['revenue'] != 0) & (df_movies['budget'] != 0), df_movies['revenue'] / df_movies['budget'], 0)

In [18]:
# Elimina las columnas 'video','imdb_id','adult','original_title','poster_path' y 'homepage'
df_movies.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)

***

CONVERSION DE TIPO DE VARIABLES

In [19]:
df_movies.head(1)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,12.451801


In [20]:
df_movies.dtypes

belongs_to_collection            object
budget                          float64
genres                           object
id                               object
original_language                object
overview                         object
popularity                       object
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                    float64
vote_count                      float64
release_year                      int64
return                          float64
dtype: object

Se observa que las columnas 'popularity' y 'id' no son del tipo de dato correcto

In [21]:
# Cambiar el tipo de valor de la columna 'popularity' a float
df_movies[['popularity']] = df_movies[['popularity']].astype(float)

# Cambiar el tipo de valor de la columna 'id' a integer
df_movies['id'] = df_movies['id'].astype(int)

In [22]:
# Ciclo for que conprueba si los primeros valores de cada columna son del tipo string
columnas_texto = []
for columna in df_movies.columns:
    if isinstance(df_movies[columna][0], str):
        columnas_texto.append(columna)

# Muestra la lista con los nombres de las columnas que contienen 
print(columnas_texto)

# Analizo las columnas y sus valores
df_movies[columnas_texto].head(2)


['belongs_to_collection', 'genres', 'original_language', 'overview', 'production_companies', 'production_countries', 'spoken_languages', 'status', 'title']


,belongs_to_collection,genres,original_language,overview,production_companies,production_countries,spoken_languages,status,title
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",en,"Led by Woody, Andy's toys live happily in his ...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]",Released,Toy Story
1,NaN,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",en,When siblings Judy and Peter discover an encha...,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Jumanji


Se observan en cada valor de algunas columnas que ahi esctructuras de listas de diccioonarios pero almacenadas como texto
Se procede a convertir los valores al tipo de dato correcto para por utilizar de mejor forma la informacion

In [23]:
# Crea una lista con los nombres de las columnas afectadas
lista_columnas_texto = ['belongs_to_collection','genres','production_companies','production_countries','spoken_languages']

In [24]:
# Funcion que utiliza a su vez la funcion eval()
def convert_to(text):
    return eval(text)

# Ciclo for que itera la lista 'lista_columnas_texto' 
for a in lista_columnas_texto:
    df_movies[a] = df_movies[a].fillna('None')          # Cambia el valor de los valores nulos a un texto 'None' para que se puede aplicar la funcion
    df_movies[a] = df_movies[a].apply(convert_to)       # Aplico la funcion 'convert_to_dicc' a cada fila con el funcion 'apply'

In [25]:
# Ciclo for que comprueba si los primeros valores de cada columna son del tipo list o dict
columnas = []
for columna in df_movies.columns:
    primer_valor = df_movies[columna][0]
    if type(primer_valor) == list or type(primer_valor) == dict:
        columnas.append(columna)

# Muestra la lista con los nombres de las columnas que contienen texto seguido de las columnas
print(columnas)
df_movies[columnas].head(2)

['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']


,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,"{'id': 10194, 'name': 'Toy Story Collection', ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}]"
1,None,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'iso_639_1': 'en', 'name': 'English'}, {'iso..."


Se obverva que los diccionarios en cada columna almacenanan nombres y ids o standares internacionales de cada nombre. Para reducir el tamaño del archivo csv final se reemplazaran los valores de cada fila solo con los valores de la llave 'name' ya que las llaves que representan ids y standares internacionales no son necesarios en este tipo de proyecto

In [26]:
# Esta funcion retorna una lista de solo los valores de la llave name de cada diccionario
def extract_name(list_dict):
    if type(list_dict) == list and len(list_dict) > 0:
        list_temp = []
        for dicc in list_dict:
            list_temp.append(dicc['name'])
        return list_temp
    if type(list_dict) == dict:
        return list_dict['name']
    else:
        return None
    
for column in lista_columnas_texto:
    df_movies[column] = df_movies[column].apply(extract_name)

df_movies[lista_columnas_texto].head()

,belongs_to_collection,genres,production_companies,production_countries,spoken_languages
0,Toy Story Collection,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[English]
1,None,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[English, Français]"
2,Grumpy Old Men Collection,"[Romance, Comedy]","[Warner Bros., Lancaster Gate]",[United States of America],[English]
3,None,"[Comedy, Drama, Romance]",[Twentieth Century Fox Film Corporation],[United States of America],[English]
4,Father of the Bride Collection,[Comedy],"[Sandollar Productions, Touchstone Pictures]",[United States of America],[English]


In [27]:
# Establesco la columna id como indice antes cambiandole el nombre a 'IdMovie' para facilitar la union con el csv 'credits'
columna_id = df_movies['id']
df_movies.drop('id',inplace=True,axis=1)
df_movies.insert(0, 'IdMovie', columna_id)
df_movies = df_movies.sort_values('IdMovie')
df_movies = df_movies.set_index('IdMovie')

In [28]:
df_movies.head(3)

,belongs_to_collection,budget,genres,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
IdMovie,,,,,,,,,,,,,,,,,,,
2,None,0.0,"[Drama, Crime]",fi,Taisto Kasurinen is a Finnish coal miner whose...,3.860491,"[Villealfa Filmproduction Oy, Finnish Film Fou...",[Finland],1988-10-21,0.0,69.0,"[suomi, Deutsch]",Released,NaN,Ariel,7.1,44.0,1988,0.000
3,None,0.0,"[Drama, Comedy]",fi,"An episode in the life of Nikander, a garbage ...",2.292110,[Villealfa Filmproduction Oy],[Finland],1986-10-16,0.0,76.0,"[English, suomi, svenska]",Released,NaN,Shadows in Paradise,7.1,35.0,1986,0.000
5,None,4000000.0,"[Crime, Comedy]",en,It's Ted the Bellhop's first night on the job....,9.026586,"[Miramax Films, A Band Apart]",[United States of America],1995-12-09,4300000.0,98.0,[English],Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,539.0,1995,1.075


***

CSV CREDITS

In [29]:
# Utiliza la funcion de pandas read_csv para cargar el contenido del archivo credits.csv a un dataframe de pandas.
df_credits = pd.read_csv('../Dataset/credits.csv',delimiter=',')
df_credits.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [30]:
# lista con los nombres de las columnas a tratar
columnas_texto = ['cast', 'crew']
# Funcion que utiliza a su vez la funcion eval()
def convert_to(text):
    return eval(text)

# Ciclo for que itera la lista 'lista_columnas_texto' 
for a in columnas_texto:
    df_credits[a] = df_credits[a].fillna('None')          # Cambia el valor de los valores nulos a un texto 'None' para que se puede aplicar la funcion
    df_credits[a] = df_credits[a].apply(convert_to)       # Aplico la funcion 'convert_to_dicc' a cada fila con el funcion 'apply'

In [31]:
# Funcion que reduce la cantidad de diccionarios almacenados en la lista en uno solo
def reductor_dicc(lista):
    if type(lista) == list and len(lista) > 0:
        keys = list(lista[0].keys())
        dicc_temp = {}
        for key in keys:
            dicc_temp[key] = []
    
        for dicc in lista:
            for key in keys:
                dicc_temp[key].append(dicc[key])
        return dicc_temp
    else:
        return None

In [32]:
# Aplico la funcion 'reductor_dicc' en cada fila de la columna 'cast' y 'crew'
for a in ['cast','crew']:
    df_credits[a] = df_credits[a].apply(reductor_dicc)

In [33]:
df_credits

,cast,crew,id
0,"{'cast_id': [14, 15, 16, 17, 18, 19, 20, 26, 2...","{'credit_id': ['52fe4284c3a36847f8024f49', '52...",862
1,"{'cast_id': [1, 8, 2, 24, 10, 25, 26, 11, 14, ...","{'credit_id': ['52fe44bfc3a36847f80a7cd1', '52...",8844
2,"{'cast_id': [2, 3, 4, 5, 6, 9, 10], 'character...","{'credit_id': ['52fe466a9251416c75077a89', '52...",15602
3,"{'cast_id': [1, 2, 3, 4, 5, 6, 8, 10, 20, 21],...","{'credit_id': ['52fe44779251416c91011acb', '52...",31357
4,"{'cast_id': [1, 2, 3, 4, 13, 14, 15, 16, 17, 1...","{'credit_id': ['52fe44959251416c75039ed7', '52...",11862
...,...,...,...
45471,"{'cast_id': [0, 1, 2], 'character': ['', '', '...","{'credit_id': ['5894a97d925141426c00818c', '58...",439050
45472,"{'cast_id': [1002, 1008, 1007, 1005, 1015, 100...","{'credit_id': ['52fe4af1c3a36847f81e9b15', '52...",111109
45473,"{'cast_id': [6, 7, 8, 10, 11, 12, 13, 9, 14, 1...","{'credit_id': ['52fe4776c3a368484e0c8387', '52...",67758
45474,"{'cast_id': [2, 3, 4, 5, 6], 'character': ['',...","{'credit_id': ['533bccebc3a36844cf0011a7', '58...",227506


Al igual que en en caso del dataframe df_movies, los diccionarios en la columna cast y crew poseen llaves que representan ids de cada nombre, por lo que se reemplazara los valores de cada fila por la lista almacenada en la llave name.

In [34]:
# Funcion que si el valor de dicc es igual a un diccionaio, retorna solo la lista almacenada en la llave name en caso contrario retorna un valor vacio
def name_actor(dicc):
    if type(dicc) == dict: 
        lst = dicc['name']
        return lst
    else:
        return None

# Utliza la funcion apply para aplicar la funcion anterior en cada fila
df_credits['cast'] = df_credits['cast'].apply(name_actor)

In [35]:
# Funcion que si el valor dicc es un diccionario, retorna solo la lista almacenada en la llave name en caso contrario retorna un valor vacio
def name_director(dicc):
    if type(dicc) == dict:
        lst_temp = []
        for index,job in enumerate(dicc['job']):            # Solo carga en la lista lst_temp si el valor de la llave 'job' es igual a 'Director'
            if job == 'Director':                           
                lst_temp.append(dicc['name'][index])
        return lst_temp
    else:
        return None
    
#Utliza la funcion apply para aplicar la funcion anterior en cada fila
df_credits['crew'] = df_credits['crew'].apply(name_director)

In [36]:
# Establesco la columna id como indice antes cambiandole el nombre a 'IdMovie'
columna_id = df_credits['id']
df_credits.drop('id',inplace=True,axis=1)
df_credits.insert(0, 'IdMovie', columna_id)
df_credits = df_credits.sort_values('IdMovie')
df_credits = df_credits.set_index('IdMovie')

UNION DE CREDITS CON MOVIES

In [38]:
# Cambia el nombre de las columnas cast y crew por actors y directors
df_credits.rename(columns={'cast': 'actors','crew':'directors'},inplace=True)

In [39]:
# Une el dataframe 'df_credits' al dataframe 'df_movies' usando como referencia las columnas 'IdMovie'.
df_movies = pd.merge(df_movies, df_credits, on='IdMovie')

In [46]:
# Elimina la columna 'IdMovie' y resetea el index para evitar errores futuros
df_movies.reset_index(drop=True, inplace=True)

In [63]:
df_movies.isna().sum()

belongs_to_collection    40954
budget                       0
genres                    2384
original_language           11
overview                   941
popularity                   0
production_companies     11817
production_countries      6219
release_date                 0
revenue                      0
runtime                    246
spoken_languages          3774
status                      80
tagline                  25026
title                        0
vote_average                 0
vote_count                   0
release_year                 0
return                       0
actors                    2354
directors                  723
dtype: int64

Como se observa las columnas presenan varios valores vacios que se trataran a continuacion

In [48]:
# Utiliza la funcion 'to_pickle' de pandas para guardar en un archivo pickle el dataframe 'df_movies'.
df_movies.to_pickle('../Data/Data_unida.pickle')

***

AJUSTE DE LA BASE DE DATOS PARA LA API

In [71]:
df_movies = pd.read_pickle('../Data/Data_unida.pickle')

In [72]:
# Elimino columnas que no son consultadas por la api para reducir el peso del archivo pickle 
df_movies.drop(['belongs_to_collection','genres','original_language','overview','production_companies',
                'production_countries','runtime','spoken_languages','tagline',],axis=1,inplace=True)

In [73]:
# Crea nueva columna con el día de la semana en formato de texto
df_movies['release_day'] = df_movies['release_date'].dt.strftime("%A")

In [74]:
# Crea nueva columna con el mes del año en formato de texto
df_movies['release_month'] = df_movies['release_date'].dt.month

In [75]:
# Se eliminan las filas donde el valor status no sea igual a released/estrenado ya que segun la consigna se debe trabajar con las peliculas estrenadas
df_movies = df_movies[df_movies['status'] == 'Released']

# Elimino la columna status ya que todas las peliculas tiene el status de estrenadas
df_movies.drop('status',inplace=True,axis=1)

In [76]:
df_movies.dtypes

budget                  float64
popularity              float64
release_date     datetime64[ns]
revenue                 float64
title                    object
vote_average            float64
vote_count              float64
release_year              int64
return                  float64
actors                   object
directors                object
release_day              object
release_month             int64
dtype: object

In [77]:
df_movies.isnull().sum()

budget              0
popularity          0
release_date        0
revenue             0
title               0
vote_average        0
vote_count          0
release_year        0
return              0
actors           2303
directors         698
release_day         0
release_month       0
dtype: int64

Se redujo significativamente la cantidad de valores vacios al eliminar las columnas que no seran utilizadas por la API.
Actors y directors no se tratara ya que no afecta al funcionamiento de la API y se mantiene la cantidad de peliculas para consultar

In [57]:
df_movies.to_pickle('../Data/api.pickle')

***

PREPARACION DE DATAFRAME PARA LA ETADA DE MACHINE LEARNING

In [88]:
# Carga archivo guardado luego de la union
df_movies = pd.read_pickle('../Data/Data_unida.pickle')

In [114]:
df_movies.head(5)

,budget,genres,overview,popularity,revenue,title,vote_average,vote_count,release_year,return
0,0.0,"[Drama, Crime]",Taisto Kasurinen is a Finnish coal miner whose...,3.860491,0.0,Ariel,7.1,44.0,1988,0.000000
1,0.0,"[Drama, Comedy]","An episode in the life of Nikander, a garbage ...",2.292110,0.0,Shadows in Paradise,7.1,35.0,1986,0.000000
2,4000000.0,"[Crime, Comedy]",It's Ted the Bellhop's first night on the job....,9.026586,4300000.0,Four Rooms,6.5,539.0,1995,1.075000
3,0.0,"[Action, Thriller, Crime]","While racing to a boxing match, Frank, Mike, J...",5.538671,12136938.0,Judgment Night,6.4,79.0,1993,0.000000
4,11000000.0,"[Adventure, Action, Science Fiction]",Princess Leia is captured and held hostage by ...,42.149697,775398007.0,Star Wars,8.1,6778.0,1977,70.490728


In [90]:
# Elimino columnas que no aportan informacion relevante para la etapa de ML
drops = ['original_language','belongs_to_collection','tagline','production_companies','production_countries','release_date','runtime','spoken_languages','actors','directors','status']
df_movies.drop(drops,inplace=True,axis=1)

In [93]:
df_movies.isna().sum()

budget          0
genres          0
overview        0
popularity      0
revenue         0
title           0
vote_average    0
vote_count      0
release_year    0
return          0
dtype: int64

In [92]:
# Elimina las filas sin valores en las columnas 'genres'y'overview'
df_movies.dropna(subset=['genres','overview'], inplace=True)

In [235]:
unicos = []
for a in df_movies['genres']:
    if type(a) == list:
        for i in a:
            if i not in unicos:
                unicos.append(i)
print(unicos)

['Drama', 'Crime', 'Comedy', 'Action', 'Thriller', 'Adventure', 'Science Fiction', 'Animation', 'Family', 'Romance', 'Mystery', 'Music', 'Horror', 'Fantasy', 'Documentary', 'War', 'Western', 'History', 'Foreign', 'TV Movie']


In [238]:
# Crear columnas nuevas en el DataFrame para cada género único y asignar valores 1 o 0
for genre in unicos:
    df_movies[genre] = df_movies['genres'].apply(lambda x: 1 if genre in x else 0)

In [239]:
df_movies.drop('genres',inplace=True,axis=1)

In [107]:
df_movies.head(2)

IdMovie
2                  Ariel
3    Shadows in Paradise
Name: title, dtype: object

In [94]:
df_movies.to_pickle('../Data/ML.pickle')

***

MACHINE LEARNING

In [103]:
# Se carga archivo pickle con las modificaciones
df_movies = pd.read_pickle('../Data/ML.pickle')

In [104]:
# importo las dependencia necesarias
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

In [105]:
# Crea una instancia de TfidfVectorizer
model = TfidfVectorizer(max_df = 0.9)

En esta primera version de decidio entrenar el modelo con la columna "overview" ya que posee una descripcion mas acertada de la pelicula que el titulo mismo

In [98]:
# Transforma los datos de texto en una matriz TF-IDF
matrix = model.fit_transform(df_movies["overview"])

In [ ]:
# Guardo el modelo entrenado en un archivo plk
joblib.dump(vectorizer, "../Models/tfidf_model.pkl")

In [99]:
# Crea una instancia de NearestNeighbors
index = NearestNeighbors(n_neighbors=6, algorithm='kd_tree')
# Ajusta el índice a la matriz TF-IDF
index.fit(matrix)

c:\Users\Guille\anaconda3\envs\render\lib\site-packages\sklearn\neighbors\_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


NearestNeighbors(algorithm='kd_tree', n_neighbors=6)

In [ ]:
# Guardo el modelo entrenado en un archivo plk
joblib.dump(knn, "../Models/knn_model.pkl")

In [100]:
def recommend_movies(title):
    overview = df_movies[df_movies['title'] == title]['overview'].iloc[0] # Busca el valor de la columna 'overview' segun el titulo ingresado
    overview_vector = model.transform([overview])  # Transforma el título de entrada en una representación vectorial
    _, indices = index.kneighbors(overview_vector)  # Busca los vecinos más cercanos basados en la representación vectorial
    recommendations = df_movies["title"].iloc[indices[0][1:]]  # Obtiene los títulos de las películas más similares (excluyendo la película de referencia)
    return recommendations

In [101]:
# Ingresa el título de la película de referencia
movie_title = "Manuel on the Island of Wonders"  
recommendations = recommend_movies(movie_title)
print("Recomendaciones de películas:")
for movie in recommendations:
    print(movie)

Recomendaciones de películas:
Wojaczek
Snow days
Afstiros katallilo
Slaves of New York
Game Over


Version entrenanda solo con la columna "title" para la API

In [111]:
matrix = model.fit_transform(df_movies["title"])

index = NearestNeighbors(n_neighbors=6, algorithm='kd_tree')
index.fit(matrix)


def recommend_movies(title):
    title_vector = model.transform([title])  # Transforma el título de entrada en una representación vectorial
    _, indices = index.kneighbors(title_vector)  # Busca los vecinos más cercanos basados en la representación vectorial
    recommendations = df_movies["title"].iloc[indices[0][1:]]  # Obtiene los títulos de las películas más similares (excluyendo la película de referencia)
    return recommendations


movie_title = "Manuel on the Island of Wonders"  # Ingresa el título de la película de referencia
recommendations = recommend_movies(movie_title)
print("Recomendaciones de películas:")
for movie in recommendations:
    print(movie)

Recomendaciones de películas:
The Wonders
The Wonders
9
D.O.A.
4.3.2.1


c:\Users\Guille\anaconda3\envs\render\lib\site-packages\sklearn\neighbors\_base.py:504: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: using brute force")


In [113]:
# Guardo el modelo entrenado en un archivo plk
joblib.dump(index, "../Models/knn_model.pkl")

# Guardo el modelo entrenado en un archivo plk
joblib.dump(model, "../Models/tfidf_model.pkl")

['../Models/tfidf_model.pkl']